# Translate isoforms to get protein products

In [23]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import gffutils
import pybedtools
import re
import numpy as np

v19db_filename = '/projects/ps-yeolab/genomes/hg19/gencode/v19/gencode.v19.annotation.gtf.db'
v19db = gffutils.FeatureDB(v19db_filename)

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
figure_folder = '/home/obotvinnik/Dropbox/figures2/singlecell_pnms/isoform_protein_properties'
! mkdir $figure_folder

mkdir: cannot create directory `/home/obotvinnik/Dropbox/figures2/singlecell_pnms/isoform_protein_properties': File exists


In [13]:
splicing_feature_data = pd.read_csv('{}/splicing_feature_data.csv'.format(folder), index_col=0)

/home/obotvinnik/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (11,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [30]:
splicing_feature_data.head()

,criteria,criteria_additional,criteria_full,ensembl_id,exon1,exon1_length,exon2,exon2_length,exon3,exon3_length,...,junction_exons12,junction_exons13,junction_exons23,junction_exons24,junction_exons34,strand,exon2_divisible_by_3,one_ensembl_id,splice_type,exon2_merkin2012_ancient
event_id,,,,,,,,,,,,,,,,,,,,,
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,99,exon:chr10:100189330-100189399:-,70,...,chr10:100189647-100190327:-,chr10:100189400-100190327:-,chr10:100189400-100189547:-,NaN,NaN,-,True,ENSG00000107521,SE,False
exon:chr10:100193697-100193848:-@exon:chr10:100190888-100191048:-@exon:chr10:100190328-100190427:-,only one,NaN,only one,ENSG00000107521,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,161,exon:chr10:100190328-100190427:-,100,...,chr10:100191049-100193696:-,chr10:100190428-100193696:-,chr10:100190428-100190887:-,NaN,NaN,-,False,ENSG00000107521,SE,False
exon:chr10:100195392-100195529:-@exon:chr10:100195029-100195171:-@exon:chr10:100193697-100193848:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,143,exon:chr10:100193697-100193848:-,152,...,chr10:100195172-100195391:-,chr10:100193849-100195391:-,chr10:100193849-100195028:-,NaN,NaN,-,False,ENSG00000107521,SE,False
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,only one,NaN,only one,ENSG00000120053,exon:chr10:101165513-101165617:-,105,exon:chr10:101163481-101163631:-,151,exon:chr10:101163226-101163391:-,166,...,chr10:101163632-101165512:-,chr10:101163392-101165512:-,chr10:101163392-101163480:-,NaN,NaN,-,False,ENSG00000120053,SE,False
exon:chr10:101419263-101419345:+@exon:chr10:101419619-101419721:+@exon:chr10:101421203-101421385:+,only one,NaN,only one,ENSG00000198018,exon:chr10:101419263-101419345:+,83,exon:chr10:101419619-101419721:+,103,exon:chr10:101421203-101421385:+,183,...,chr10:101419346-101419618:+,chr10:101419346-101421202:+,chr10:101419722-101421202:+,NaN,NaN,+,False,ENSG00000198018,SE,False


## Translate isoform

In [58]:
prefix = 'isoform_translations'
translated_fasta = '{}/{}.fa'.format(folder, prefix)

In [98]:
# %%time

import itertools
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio.Alphabet import generic_dna, generic_protein

hg19_fasta = '/projects/ps-yeolab/genomes/hg19/gencode/v19/GRCh37.p13.genome.fa'

splice_type_isoform_exons = {'SE': {'isoform1': ('exon1', 'exon3'),
                                    'isoform2': ('exon1', 'exon2', 'exon3')},
                             'MXE': {'isoform1': ('exon1', 'exon3', 'exon4'),
                                     'isoform2': ('exon1', 'exon2', 'exon4')}
                            }

# splice_type_isoform1_exons = {'SE': ('exon1', 'exon3'), 
#                               'MXE': ('exon1', 'exon3', 'exon4')}
# splice_type_isoform2_exons = {'SE': ('exon1', 'exon2','exon3'), 
#                               'MXE': ('exon1', 'exon2', 'exon4')}

best_tags = 'appris_principal', 'appris_candidate', 'CCDS', 'exp_conf', 'basic'

def overlap(x, y):
    return not ((x.start > y.stop) or (x.stop < y.start))


def filter_on_tags(features, tags):
    if len(features) == 0:
        return None
    
    if len(features) == 1:
        return features.pop()
    
    feature_tags = map(lambda x: (x, tuple(x.attributes['tag'])) 
                    if 'tag' in x.attributes else (x, ()), features)
#     print 'feature_tags', feature_tags
    for tag in tags:
        for feature, feature_tag in feature_tags:
            if tag in feature_tag:
                return feature
    # If none of the tags are there, return a random feature
#     print features
    return np.random.choice(list(features))

seqrecords = []

for i, (event_id, row) in enumerate(splicing_feature_data.iterrows()):
#     if i > 10:
#         break
    
    if (i+1) % 1000 == 0:
        print i+1
#     exon1 = v19db[row.exon1]
#     exon2 = v19db[row.exon2]
#     exon3 = v19db[row.exon3]
#     print event_id
    
#     exon_trio = exon1, exon2, exon3

    isoform1_exons = splice_type_isoform_exons[row.splice_type]['isoform1']
    isoform2_exons = splice_type_isoform_exons[row.splice_type]['isoform2']
    
    isoform_to_exons = {'isoform1': map(lambda x: v19db[row[x]], isoform1_exons),
                        'isoform2': map(lambda x: v19db[row[x]], isoform2_exons)}
    
    isoform_to_transcripts = dict((k, set.intersection(*map(lambda x: set(v19db.parents(x, featuretype='transcript')), v)))
                                  for k, v in isoform_to_exons.items())
    
#     isoform_to_exons = {'isoform1': (exon1, exon3), 'isoform2': exon_trio}
    
    
    # Make sure isoform1 transcripts are distinct from isoform2
    isoform1_transcripts = isoform_to_transcripts['isoform1'] - isoform_to_transcripts['isoform2']
#     print '\tisoform1_transcripts', ','.join(map(lambda x: x.id, isoform1_transcripts))
#     print '\t', '\t'.join(map(lambda x: str(list(x.attributes['tag'])) if 'tag' in x.attributes else '-', isoform1_transcripts))
    
    # Make sure isoform2 transcripts are distinct from isoform1
    isoform2_transcripts = isoform_to_transcripts['isoform2']
    if row.splice_type == 'MXE':
        isoform2_transcripts = isoform2_transcripts - isoform1_transcripts
#     print '\tisoform2_transcripts', ','.join(map(lambda x: x.id, isoform2_transcripts))
#     print '\t', '\t'.join(map(lambda x: str(list(x.attributes['tag'])) if 'tag' in x.attributes else '-', isoform2_transcripts))
    
    isoform1_transcript = filter_on_tags(isoform1_transcripts, best_tags)
    isoform2_transcript = filter_on_tags(isoform2_transcripts, best_tags)
    
#     if isoform1_transcript is not None:
#         print '\t--isoform1', isoform1_transcript.id,
#         if 'tag' in isoform1_transcript.attributes:
#             print isoform1_transcript.attributes['tag']
#         else:
#             print
#     if isoform2_transcript is not None:
#         print '\t--isoform2', isoform2_transcript.id,
#         if 'tag' in isoform2_transcript.attributes:
#             print isoform2_transcript.attributes['tag']
#         else:
#             print
    
#     isoforms = {'isoform1': isoform1_transcripts, 
#                 'isoform2': isoform2_transcripts}
    isoforms = {'isoform1': isoform1_transcript, 
                'isoform2': isoform2_transcript}
    
    
#     isoform_to_cds = {'isoform1': [], 'isoform2': []}
    for isoform, transcript in isoforms.items():
        if transcript is None:
#             print '\t\t', isoform, 'is None'
            continue
        exons = isoform_to_exons[isoform]
#         print '\t', '\t'.join(map(lambda x: str(list(x.attributes['tag'])) if 'tag' in x.attributes else '-', exons))
        
#         for transcript in transcripts:


        reverse = transcript.strand == '-'
        cdss = v19db.children(transcript, featuretype='CDS', order_by='start', 
                              reverse=reverse)

        cdss = filter(lambda cds: any(map(lambda exon: overlap(cds, exon), exons)), cdss)
#         print cdss
        if len(cdss) == len(exons):
            cds_str = '@'.join(map(lambda x: x.id, cdss))
#                 if cds_str in isoform_to_cds[isoform]:
#                     continue

#                 isoform_to_cds[isoform].append(cds_str)

#                 print '\t\t', '\t'.join(map(lambda x: x.id, exons))
#                 print '\t\t\t', '\t'.join(map(lambda x: x.id, cdss))
#                 if 'tag' in transcript.attributes:
#                     print '\t\t\t', transcript.attributes['tag']
#                 else:
#                     print '\t\t\t-'

            if reverse:
                coding_sequence = Seq(''.join(cds.sequence(hg19_fasta)[::-1] for cds in cdss), generic_dna).complement()
            else:
                coding_sequence = Seq(''.join(cds.sequence(hg19_fasta) for cds in cdss), generic_dna)
#                 print len(coding_sequence)
            coding_sequence = coding_sequence[int(cdss[0].frame):]
            translated = coding_sequence.translate()
            seqrecord = SeqRecord(translated, id='{0}|{1}|{2}'.format(event_id, cds_str, isoform))
#             print '\t\t', translated
            seqrecords.append(seqrecord)
#     pprint(isoform_to_cds)




1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000


In [100]:
len(seqrecords)

22408

In [101]:
with open(translated_fasta, 'w') as f:
    SeqIO.write(seqrecords, f, 'fasta')

In [102]:
! wc -l $translated_fasta

75638 /projects/ps-yeolab/obotvinnik/singlecell_pnms/isoform_translations.fa


In [2]:
import itertools
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
import pandas as pd

translated_df = pd.DataFrame(columns=['isoform1', 'isoform2'])

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:784: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())


In [6]:
folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

In [7]:
prefix = 'isoform_translations'
translated_fasta = '{}/{}.fa'.format(folder, prefix)

In [8]:
! wc -l $translated_fasta

75638 /projects/ps-yeolab/obotvinnik/singlecell_pnms/isoform_translations.fa


In [9]:
with open(translated_fasta) as f:
    for record in SeqIO.parse(f, 'fasta'):
        event_id, cds_str, isoform = record.id.split('|')
        translated_df.loc[event_id, isoform] = str(record.seq)

In [10]:
translated_df.shape

(18767, 2)

In [107]:
translated_df.head()

,isoform1,isoform2
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,NaN,HSASSLRPADLLALILLVQDLYPSESTAEDDIQPSPRRARSSQNIP...
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,NaN,NAPEFSIVVLHACAHNPTGIDPTPEQWKQIASVMKHRFLFPFFDSA...
exon:chr10:101421203-101421385:+@exon:chr10:101439018-101439223:+@exon:chr10:101439482-101439632:+,NaN,ISFSYLCPASWYFTVPTVSPFLRQRVAFLGLFFISCLLLLMLIIDF...
exon:chr10:101480744-101480825:-@exon:chr10:101478103-101478257:-@exon:chr10:101476105-101476218:-,NaN,LPETHQLLQLRRFAHGTAGLVFLTALSGAFVAGLDAGLVYNSFPKM...
exon:chr10:101507014-101507147:+@exon:chr10:101510126-101510153:+@exon:chr10:101514286-101514391:+,FDMVHDPMAALETLLTLGFERVLTSGCDSSALEGLPLIKRLIEQEVV,FDMVHDPMAALETLLTLGFERVLTSGCDSSALEGLPLIKRLIEQAK...


In [110]:
translated_df.dropna().shape

(3641, 2)

In [11]:
translated_df.columns = [x + '_translation' for x in translated_df]

In [15]:
if len(splicing_feature_data.columns.intersection(translated_df.columns)) == 0:
    splicing_feature_data = splicing_feature_data.join(translated_df)
splicing_feature_data.head()

,criteria,criteria_additional,criteria_full,ensembl_id,exon1,exon1_length,exon2,exon2_length,exon3,exon3_length,...,isoform1_gc,isoform1_gc_position1,isoform1_gc_position2,isoform1_gc_position3,isoform2_gc,isoform2_gc_position1,isoform2_gc_position2,isoform2_gc_position3,isoform1_translation,isoform2_translation
event_id,,,,,,,,,,,,,,,,,,,,,
exon:chr10:100190328-100190427:-@exon:chr10:100189548-100189646:-@exon:chr10:100189330-100189399:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100190328-100190427:-,100,exon:chr10:100189548-100189646:-,99,exon:chr10:100189330-100189399:-,70,...,75.714286,87.500000,77.551020,60.465116,77.292576,89.610390,73.076923,68.918919,NaN,HSASSLRPADLLALILLVQDLYPSESTAEDDIQPSPRRARSSQNIP...
exon:chr10:100193697-100193848:-@exon:chr10:100190888-100191048:-@exon:chr10:100190328-100190427:-,only one,NaN,only one,ENSG00000107521,exon:chr10:100193697-100193848:-,152,exon:chr10:100190888-100191048:-,161,exon:chr10:100190328-100190427:-,100,...,79.523810,88.311688,74.242424,74.626866,79.178886,73.333333,89.256198,73.913043,NaN,NaN
exon:chr10:100195392-100195529:-@exon:chr10:100195029-100195171:-@exon:chr10:100193697-100193848:-,best,appris_principal,"best,appris_principal",ENSG00000107521,exon:chr10:100195392-100195529:-,138,exon:chr10:100195029-100195171:-,143,exon:chr10:100193697-100193848:-,152,...,78.111588,81.927711,73.239437,78.481013,75.882353,75.206612,53.684211,93.548387,NaN,NaN
exon:chr10:101165513-101165617:-@exon:chr10:101163481-101163631:-@exon:chr10:101163226-101163391:-,only one,NaN,only one,ENSG00000120053,exon:chr10:101165513-101165617:-,105,exon:chr10:101163481-101163631:-,151,exon:chr10:101163226-101163391:-,166,...,68.899522,62.318841,70.588235,73.611111,71.246006,70.642202,58.585859,83.809524,NaN,NAPEFSIVVLHACAHNPTGIDPTPEQWKQIASVMKHRFLFPFFDSA...
exon:chr10:101419263-101419345:+@exon:chr10:101419619-101419721:+@exon:chr10:101421203-101421385:+,only one,NaN,only one,ENSG00000198018,exon:chr10:101419263-101419345:+,83,exon:chr10:101419619-101419721:+,103,exon:chr10:101421203-101421385:+,183,...,74.111675,74.242424,76.271186,72.222222,70.833333,66.666667,71.276596,75.000000,NaN,NaN


In [16]:
splicing_feature_data.to_csv('{}/splicing_feature_data.csv'.format(folder))

(3641, 2)